# Analise  Expansão de Consultas
### Autor: Vinicius Brandão Araújo
#### Considerando a base de dados referentes as noticias do portal Estadão relativos as eleições, temos o intuito de responder as seguintes questões utilizando algoritimos para epansão de consultas. 
    1. Quais os termos retornados para a expansão de cada consulta?
    2. Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
    3. Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
    4. A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

#### Assim, utilizamos na analise abaixo os algoritimos de busca expansiva e uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).

In [2]:
import pandas as pd
import numpy as np
import nltk
import scipy.sparse as sps
import re
from scipy import sparse
from nltk import bigrams    
from unicodedata import normalize
from nltk.corpus import stopwords
from collections import Counter

Em primeiro momento vamos conhecer os dados que consiste em um .csv de notícias do Estadão. Temos no arquivo acesso ao timestamp, título da notícia, subtítulo da notícias, o conteúdo da notícia, a url e, por último, o id

In [3]:
leitura = pd.read_csv('date/estadao_noticias_eleicao.csv')
leitura

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",5
5,2014-12-31T00:00:00Z,NaN,Veja as principais fotos do dia e dos eventos ...,NaN,"http://fotos.estadao.com.br/fotos/politica,dil...",6
6,2014-12-31T00:00:00Z,Veja os desafios dos governadores que assumem ...,NaN,"No Acre, governador reeleito quer erradicar an...","http://politica.estadao.com.br/noticias/geral,...",7
7,2014-12-31T00:00:00Z,PT impulsiona cerimônia de posse da Dilma nas ...,"Já entre os 27 governadores eleitos, apenas se...","Os perfis da presidente Dilma Rousseff, nas re...","http://politica.estadao.com.br/noticias/geral,...",8
8,2014-12-31T00:00:00Z,NaN,"Em setembro de 2014, após conceder entrevista ...",NaN,"http://tv.estadao.com.br/videos,politica,dilma...",9
9,2014-12-31T00:00:00Z,Dilma veta projeto de Suplicy que cria linha o...,Proposta havia sido aprovada no Senado depois ...,A presidente Dilma Rousseff vetou integralment...,"http://politica.estadao.com.br/noticias/geral,...",10


### Função para limpar o texto
Essa Função remove todos os caracteres especiais do texto bem como sua acentuação.

In [4]:
def text_clear(text):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', text)

### Lista de stopwords
Criando lista de stopwords à ser removida da matrix de coocorrência

In [5]:
words = [text_clear(stopword) for stopword in stopwords.words('portuguese')]

### Join do conteúdo
Juntando os títulos das notícias com seus respectivos subtítulos e conteúdos, e também removendo das nóticias os caracteres especiais e a acentuação para posteriomente facilitar a tokenização.

In [6]:
conteudos = leitura.titulo + " " + leitura.subTitulo +  " " + leitura.conteudo
conteudos  = conteudos.fillna("")
conteudos = conteudos.apply(text_clear)
ids = leitura.idNoticia

### Tokenizando conteúdo
Criando tokens com cada palavra do texto para que posteriormente possam ser indexadas e associadas aos respectivos ids das notícias, e contando a frequência de cada termo no texto.

In [7]:
noticias = conteudos.apply(nltk.word_tokenize)
freq_term = noticias.apply(Counter)

In [8]:
tokens_lists = conteudos.apply(lambda text: text.lower().split())
noticias_token = [token for tokens_list in tokens_lists for token in tokens_list if token not in words]

#### Criação do Index dos termos.
Para facilitar o processo, armazenaremos os termos em um dicionario e nela associamos os documentos que contem esses termos. 

In [9]:
index = {}
for i in range(len(noticias)):
    id_noticia = ids[i]
    for palavra in noticias[i]:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = {}
        id_rec = index[palavra].get(id_noticia)
        
        if not id_rec:
            docs = index[palavra]
            docs[id_noticia] = freq_term[i][palavra]

### Função Geradora de Pesos
Função responsavel por  gerar um vetor os pesos para os doc de acordo com o tf dos termos.

In [10]:
def generator_docs_peso(termos):
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = index[termo]
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else tf for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                doc_vector[i] = tf
    return docs_peso

In [11]:
def generator_query(phase):
    query = np.array([1 if index.get(termo) else 0 for termo in phase])
    return query

### Função de Busca TF
Método responsavel por gerar uma busca de termo baseado em seu tf

In [12]:
def seach_tf(phase):
    docs_tf = generator_docs_peso(phase)
    query = generator_query(phase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True) 
    return [doc[0] for doc in doc_rank]

### Bigrama
A função a seguir foi desenvolvida por Allan Sales e pode ser encontrada [aqui](https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb). 

In [13]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [14]:
matrix, vocab =co_occurrence_matrix(noticias_token)

In [15]:
matrix_consult = matrix.tocsr()

### Consulta Frequencia
Método que utiliza a matrix do termose retornando a frequencia dos mesmo. 

In [16]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

### Get Ocorrencia
Método que retornar as palavras rankiadas em 3 que mais estão associadas ao termo pesquisado. 

In [17]:
def get_ocorrence(palavra):
    list_occurency = matrix_consult[vocab[palavra]].getrow(0).toarray()[0]
    indexs, frequency = zip(*sorted(enumerate(list_occurency), key=lambda x: x[1], reverse=True))
    return indexs[:3], frequency[:3]

## Busca por expansão.
O método é responsavel por retornar a busca por expansão do termo que é passado assim ele gera as palavras que mais se associa aquele terno e retornar os documentos associados.

In [18]:
def seach_expansao(termo):
    ocorrencias = get_ocorrence(termo)
    expansao = [word for key in ocorrencias[0] for word in vocab.keys() if vocab[word] == key]
    expansao.append(termo)
    doc_exp = seach_tf(expansao)
    return expansao, doc_exp

# Analises referentes a Busca Expansiva 
## Para está analise utilizaremos a busca dos seguintes termos :
  ### 1. Dilma
  ### 2. Aercio
  ### 3. Lula
## Utilizamos o codigo abaixo para gerar os resultados da tabela:
          termos = lula,dilma,aercio
          expansao, doc_exp = seach_expansao(termos)
          print(expansao)
          print(len(doc_exp))
          print(len(index[termos]))

Termo | Termo Expansão | Busca Original | Busca Expansão
--- | --- | ---
Dilma | rousseff, disse, aecio, dilma| 2669 | 3853
Aecio| neves, disse, afirmou, aecio | 1578 | 3658
Lula| silva, dilma, disse, lula | 1249 | 3974

### Quais os termos retornados para a expansão de cada consulta?

* Dilma: 
    * rousseff
    * disse 
    * aecio 
    * dilma
* Aecio: 
    * neves
    * disse 
    * afirmou
    * aecio 
* Lula: 
    * silva
    * dilma
    * disse 
    * lula

### Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
— Sim, podemos observar, por exemplo, que a palavra que mais aparece nas três consultas refere-se ao sobrenome das pessoas que são utilizadas como termo e assim podemos verificar que nas notícias os nomes referidos vêm sempre junto ao seu sobrenome e assim é de fato relacionados com a consulta original. Vejamos que a palavra disse repete-se na consulta dos três termos e como trata das notícias relacionadas a eleições podemos verificar que é algo recorrente nas entrevistas dos políticos.

### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
Em comparação com a quantidade de documentos retornados pela consulta original e a expandida (Dados contidos na tabela acima) podemos observar que a busca por expansão captura mais documentos, assim consideramos que a busca expansiva captura mais a necessidade de informação como podemos observar de forma direta o número de documentos maior aumenta a chance de encontrar o documento necessário que o usuário busca e assim a busca retoria com mais eficiência o documento certo, mas é claro que junto a está musica expansiva é bom associar um rankeamento para ajudar ainda mais na precisão dos documentos retornados pós na ausência o usuário pode passar mais tempo procurando o documento desejado já que na busca expansiva retornar mais documentos.


### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?
Com a expansão da consulta o recall ira melhorar pois ele é a razão do total de documentos relevantes, assim temos que quanto maior for a quantidade de documentos retornados maior será a quantidade de documentos relevantes que estarão contidos nessa busca, desse modo implica que quanto maior o recall mas a precisão diminui já que ela é a razão entre a quantidade de documentos relevantes na busca e a quantidade de documentos totais presentes na busca. Podemos observar esse fato na formula Formula do recall: $\frac{tp}{tp+fn}$; formula do precision: $\frac{tp}{tp+fp}$; onde, tp é o total de documentos relevantes na busca, fn é o total de documentos relevantes não presentes na busca e fp é o total de documentos não relevantes na busca.